In [ ]:
%load_ext autoreload
%autoreload 2
from datetime import datetime
from pathlib import Path
import matplotlib.pyplot as plt
import tensorflow as tf
from watch_recognition.models import build_backbone
from watch_recognition.reports import run_on_image_debug, log_scalar_metrics
from watch_recognition.data_preprocessing import load_keypoints_data_as_kp
from watch_recognition.augmentations import set_shapes, process_kp_data, encode_keypoints_to_mask
from functools import partial
from watch_recognition.augmentations import view_image

plt.style.use("dark_background")
%matplotlib inline


ROOT_DIR = ".."
EPOCHS = 30
image_size = (224, 224)

In [ ]:
base_model = tf.keras.applications.EfficientNetB0(
    weights="imagenet",  # Load weights pre-trained on ImageNet.
    input_shape=(*image_size, 3),
    include_top=False,
)
for layer in base_model.layers:
    if 'project_conv' in layer.name:
        print(layer.name, layer.output.shape)

In [ ]:
base_model = build_backbone(image_size)
# base_model.trainable = False


In [ ]:
# build model

inputs = tf.keras.Input(shape=(*image_size, 3), )
x = base_model(inputs)
for i in range(2):
    x = tf.keras.layers.Conv2D(
        filters=128, kernel_size=3, padding="same", activation=None
    )(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation(tf.keras.activations.swish)(x)

x = tf.keras.layers.UpSampling2D()(x)
for i in range(2):
    x = tf.keras.layers.Conv2D(
        filters=128, kernel_size=3, padding="same", activation=None
    )(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation(tf.keras.activations.swish)(x)
# x = tf.keras.layers.Dropout(0.3)(x)
tf.keras.layers.Flatten()
output = tf.keras.layers.Conv2D(
    filters=4, kernel_size=1, strides=1, padding="same", activation='softmax'
)(x)

model = tf.keras.models.Model(
    inputs=inputs, outputs=output
)

In [ ]:
model.summary()


In [ ]:
model.output.shape[1:]

In [ ]:
X, y, _ = load_keypoints_data_as_kp(
    Path("%s/download_data/keypoints/train" % ROOT_DIR),
)
X.shape, y.shape

In [ ]:

encode_kp = partial(encode_keypoints_to_mask, image_size=(224, 224, 3),
                    mask_size=(28, 28),
                    extent=(1, 1),
                    separate_hour_and_minute_handles=False,
                    )
set_shape_f = partial(set_shapes, img_shape=(224, 224, 3), target_shape=(28, 28, 1))

In [ ]:


from watch_recognition.augmentations import add_sample_weights

dataset = tf.data.Dataset.from_tensor_slices((X, y))
AUTOTUNE = tf.data.experimental.AUTOTUNE

ds_alb = dataset.map(
    process_kp_data,
    num_parallel_calls=AUTOTUNE,
)

ds_alb = ds_alb.map(
    encode_kp,
    num_parallel_calls=AUTOTUNE
)
# Try changing the order to shuffle, batch, map, prefetch
ds_alb = ds_alb\
    .map(set_shape_f, num_parallel_calls=AUTOTUNE)\
    .map(add_sample_weights)\
    .shuffle(8 * 32)\
    .batch(32)\
    .prefetch(AUTOTUNE)
ds_alb


In [ ]:
view_image(ds_alb)

In [ ]:
X_val, y_val, _ = load_keypoints_data_as_kp(
    Path("%s/download_data/keypoints/validation" % ROOT_DIR),
)
X_val.shape, y_val.shape

In [ ]:
dataset_val = tf.data.Dataset.from_tensor_slices((X_val, y_val))
dataset_val = dataset_val\
    .map(encode_kp, num_parallel_calls=AUTOTUNE)\
    .map(set_shape_f, num_parallel_calls=AUTOTUNE)\
    .map(add_sample_weights)\
    .batch(X_val.shape[0])\
    .prefetch(AUTOTUNE)\
    .cache()
dataset_val

In [ ]:
view_image(dataset_val)


In [ ]:
optimizer = tf.keras.optimizers.Adam(3e-4)
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    optimizer=optimizer,
)

start = datetime.now()
TYPE = 'keypoint'
MODEL_NAME = 'efficientnetb0'
logdir = f"tensorboard_logs/{TYPE}/{MODEL_NAME}/run_{start.timestamp()}"
print(logdir)
file_writer_distance_metrics_train = tf.summary.create_file_writer(logdir + "/train")
file_writer_distance_metrics_validation = tf.summary.create_file_writer(
    logdir + "/validation")

model_path = f'models/{TYPE}/{MODEL_NAME}/run_{start.timestamp()}.h5'
model.fit(
    ds_alb,
    epochs=EPOCHS,
    validation_data=dataset_val,
    callbacks=[
        tf.keras.callbacks.TensorBoard(
            log_dir=logdir,
            update_freq="epoch",
        ),
        tf.keras.callbacks.ModelCheckpoint(
            filepath=model_path,
            save_weights_only=False,
            monitor='val_loss',
            save_best_only=True,
        ),
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.8,
            patience=20,  # todo change to 10
            min_lr=1e-6,
            cooldown=3,
            verbose=1,
        ),
        tf.keras.callbacks.LambdaCallback(
            on_epoch_end=partial(log_scalar_metrics, X=X, y=y,
                                 file_writer=file_writer_distance_metrics_train, model=model)),
        tf.keras.callbacks.LambdaCallback(
            on_epoch_end=partial(log_scalar_metrics, X=X_val, y=y_val,
                                 file_writer=file_writer_distance_metrics_validation, model=model)),

    ]
)
elapsed = (datetime.now() - start).seconds
print(
    f"total training time: {elapsed / 60} minutes, average: {elapsed / 60 / EPOCHS} minutes/epoch")


In [ ]:
train_X, train_y, _ = next(iter(ds_alb))
train_X, train_y = train_X.numpy(), train_y.numpy()
run_on_image_debug(model, train_X[0])

In [ ]:
for image in X_val[10:20]:
    run_on_image_debug(model, image)

In [ ]:
model.save(model_path + "-end")

In [ ]:
path = Path(f"{ROOT_DIR}/example_data/Zrzut ekranu 2021-08-25 o 22.24.14.jpg")
test_image = tf.keras.preprocessing.image.load_img(
    path, "rgb", target_size=image_size, interpolation="bicubic",
)
test_image_np = tf.keras.preprocessing.image.img_to_array(test_image)

In [ ]:
run_on_image_debug(model, test_image_np)

In [ ]:
# model_path = "./models/keypoint/efficientnetb0/run_1631135030.876396.h5"
print(model_path)
loaded_model = tf.keras.models.load_model(model_path, compile=False)

In [ ]:
for path in [
    Path(f"{ROOT_DIR}/example_data/Zrzut ekranu 2021-08-25 o 22.24.14.jpg"),
    Path(f"{ROOT_DIR}/example_data/Zrzut ekranu 2021-08-25 o 22.24.24.jpg"),
    Path(f"{ROOT_DIR}/example_data/test-image-2.jpg"),
]:
    test_image = tf.keras.preprocessing.image.load_img(
        path, "rgb", target_size=image_size, interpolation="bicubic",
    )
    test_image_np = tf.keras.preprocessing.image.img_to_array(test_image)

    run_on_image_debug(loaded_model, test_image_np)

In [ ]:
for image in X_val[10:20]:
    run_on_image_debug(loaded_model, image)


In [ ]:

for image in X[10:20]:
    run_on_image_debug(loaded_model, image)